### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Duchenne_Muscular_Dystrophy/GSE79263'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Ensure sample_characteristics is defined. This should be obtained in previous steps.
sample_characteristics = {
    0: ['cell type: non-myogenic CD56-negative', 'differentiation state: Myoblast', 'differentiation state: Myotube'],
    1: ['differentiation state: NA', 'clonal state: Clone', 'clonal state: Primary'],
    2: ['clonal state: NA', 'disease state: healthy', 'disease state: Duchenne muscular dystrophy', 'disease state: Healthy'],
    3: ['disease state: NA', nan],
    4: ['age: 80y', 'age: 78y', 'age: unknown', 'age: 79y', 'age: 19y', 'age: 17y', 'age: 15y', 'age: 73y', nan]
}

# Assuming clinical_data is defined or passed appropriately from previous steps
clinical_data = ...

# Step 1: Determine if the dataset likely contains gene expression data
is_gene_available = True

# Step 2: Variable Availability and Data Type Conversion

# Duchenne_Muscular_Dystrophy
trait_values = sample_characteristics[2] if 2 in sample_characteristics else []
if len(set(trait_values)) > 1:
    trait_row = 2

# Age
age_values = sample_characteristics[4] if 4 in sample_characteristics else []
if len(set(age_values)) > 1:
    age_row = 4

# Gender
# No gender information in the provided sample characteristics dictionary
gender_row = None

# Conversion Functions
def convert_trait(value):
    if value:
        val = value.split(':')[-1].strip().lower()
        if val == 'duchenne muscular dystrophy':
            return 1
        elif val == 'healthy':
            return 0
    return None

def convert_age(value):
    if value and ':' in value:
        val = value.split(':')[-1].strip().lower()
        if val.endswith('y'):
            try:
                return int(val.replace('y', ''))
            except ValueError:
                return None
    return None

def convert_gender(value):
    return None

# Step 3: Save Metadata
save_cohort_info('GSE79263', './preprocessed/Duchenne_Muscular_Dystrophy/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Duchenne_Muscular_Dystrophy', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Duchenne_Muscular_Dystrophy/trait_data/GSE79263.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
